### To run this model yourself: go to Run -> Run All Cells in the top left menu bar.

In [1]:
from epx import Job, ModelConfig, SynthPop

import pandas as pd
import numpy as np
import json
import time

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import plotly.io as pio
from plotly.subplots import make_subplots

from epxexec.visual.utils import default_plotly_template
pio.templates["epistemix"] = default_plotly_template()
pio.templates.default = "epistemix"

mapstyle="mapbox://styles/pnowell/cl4n9fic8001i15mnfmozrt8j"
token="pk.eyJ1IjoicG5vd2VsbCIsImEiOiJja201bHptMXkwZnQyMnZxcnFveTVhM2tyIn0.Pyarp9gHCON4reKvM2fZZg"

# Epstein Rebellion model applied to Grand Isle County, VT

This model captures the evolution of civil violence in a population split into regular agents, who are disaffected and willing to rebel if the conditions are right, and cop agents, who try to quell rebellion by jailing those agents.

In this implementation, the households of Grand Isle County serve as the locations between which agents move, but the agent behaviors are otherwise unchanged from the grid implementation.

### Key simulation variables

- number of locations: 2939
- agent density: 0.7
- cop density: 0.05
- government legitimacy: 0.5
- max jail sentence: 20 days
- agent vision: 150 closest agents
- rebellion threshold: 0.1

## Place and agent generation

In [2]:
### FIPS code for county of choice: Grand Isle VT
### change this to generate agent files for a different location

fips = "50013"

In [3]:
import os
path_to_pop = os.getenv('FRED_DATA') + '/country/usa/US_2010.v5/'+fips+"/"

households = pd.read_csv(path_to_pop+"household.txt")
workplaces = pd.read_csv(path_to_pop+"workplace.txt")
schools = pd.read_csv(path_to_pop+"school.txt")

In [4]:
import math


def haversine_delta_long(center, long):
    lon1, lat1 = center
    lon2 = long
    lat2 = lat1
    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers
    
    if lon2 < lon1:
        return -km
    else:
        return km

def haversine_delta_lat(center, lat):
    lon1, lat1 = center
    lon2 = lon1
    lat2 = lat
    
    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers
    
    if lat2 < lat1:
        return -km
    else:
        return km

In [5]:
center = (households.LON.median(), households.LAT.median())

households = households.assign(xloc=[haversine_delta_long(center, x) for x in households.LON])
households = households.assign(yloc=[haversine_delta_lat(center, x) for x in households.LAT])

In [6]:
tot_places = len(households)

place_agents = pd.DataFrame()
place_agents["ID"] = np.arange(1,tot_places+1)
place_agents["x_pos"] = households.xloc
place_agents["y_pos"] = households.yloc
place_agents["agent_type"] = 1

agent_density = 0.7
cop_density = 0.05

n_regular_agents = int(agent_density*tot_places)
n_cop_agents = int(cop_density*tot_places)

regular_id_int = 100000
regular_agents = pd.DataFrame()
regular_agents["ID"] = np.arange(regular_id_int+1,regular_id_int+n_regular_agents+1)
regular_agents["agent_type"] = 2

cop_id_int = 200000
cop_agents = pd.DataFrame()
cop_agents["ID"] = np.arange(cop_id_int+1,cop_id_int+n_cop_agents+1)
cop_agents["agent_type"] = 3

place_agents.to_csv("county_places.csv", index=False)
regular_agents.to_csv("county_agents.csv", index=False)
cop_agents.to_csv("county_cops.csv", index=False)

place_agents["lat"] = households["LAT"]
place_agents["lon"] = households["LON"]

## Running the Model

In [17]:
epstein_rebellion_geo_job.status.logs

,run_id,level,time,message
0,0,INFO,2024-05-21 13:44:32.241,Environment variable FRED_DATA=/data/data
1,0,INFO,2024-05-21 13:44:32.256,Environment variable FRED_LIBRARY=/
2,0,INFO,2024-05-21 13:44:32.256,FRED simulation seed: 13649620900944172275
3,0,INFO,2024-05-21 13:44:32.370,Estimated total agents = 6965
4,0,INFO,2024-05-21 13:44:33.065,About to simulate 91 days.
5,0,INFO,2024-05-21 13:44:33.065,"Beginning to simulate day 0, which is Sun 2023..."
6,0,ERROR,2024-05-21 13:44:33.308,Error read_agent_file(county_places.csv): agen...
7,0,ERROR,2024-05-21 13:44:33.308,FRED exiting with code 2


In [18]:
epstein_rebellion_geo_job.delete(interactive=False)

In [19]:
epstein_rebellion_geo_config = ModelConfig(
             synth_pop=SynthPop("US_2010.v5", ["Grand_Isle_County_VT"]),
             start_date="2023-01-01",
             end_date="2023-04-01",
         )

epstein_rebellion_geo_job = Job(
    "model/rebellion.fred",
    config=[epstein_rebellion_geo_config],
    key="cl_epstein_rebellion_geo_job1",
    fred_version="10.1.1",
    results_dir="/home/epx/cl-results"
)


epstein_rebellion_geo_job.execute()

# the following loop idles while we wait for the simulation job to finish
start = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(epstein_rebellion_geo_job.status) != 'DONE':
    if time.time() > start + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    if str(epstein_rebellion_geo_job.status) == 'ERROR':
        break
    time.sleep(idle_time)

str(epstein_rebellion_geo_job.status)

'DONE'

## Visualizing the Results

In [8]:
agent_data = epstein_rebellion_geo_job.results.csv_output("regular_agent_info.csv")
movement = epstein_rebellion_geo_job.results.csv_output("movement.csv")

sim_data = pd.merge(left=movement, 
         right=agent_data, on=["id","simday"], how="left").fillna(0)

sim_data.agent_status = sim_data.agent_status.map({0:"Passive", 1:"Rebel", 2:"Cop", -1:"Jailed"})

sim_data["will_to_rebel"] = sim_data.grievance - sim_data.risk * sim_data.est_arrest_prob

dates = epstein_rebellion_geo_job.results.dates()
dates = dates.rename(columns={"sim_day":"simday"}).drop(columns=["run_id"])

sim_data = sim_data.merge(dates, on="simday", how="left")

tmp = sim_data[(sim_data.simday==0)&(sim_data.agent_status=="Passive")].copy()
tmp.simday = 1
sim_data = pd.concat([sim_data, tmp])

tmp = sim_data[(sim_data.simday==0)&(sim_data.agent_status=="Cop")].copy()
tmp.simday = 1
sim_data = pd.concat([sim_data, tmp])


dict = {"agent_status":["Cop", "Passive", "Rebel", "Jailed"], 
        "simday":[0]*4, "id":[0]*4, "xloc":[-100]*4, "yloc":[-1]*4, "agent_type":[2]*4}
sim_data = pd.concat([pd.DataFrame(dict), sim_data])

sim_data = sim_data.assign(size_val=2)

ValueError: No objects to concatenate

In [ ]:
agent_states = (sim_data[(sim_data.agent_status!="Cop")]
 .groupby(["simday", "agent_status"]).size().to_frame().reset_index()
 .rename(columns={0:"Count"})
)

fig = go.Figure()

for x in ["Passive", "Rebel", "Jailed"]:
    fig.add_trace(
        go.Scatter(
            x=agent_states[agent_states.agent_status==x].simday,
            y=agent_states[agent_states.agent_status==x].Count,
            mode="lines",
            line=go.scatter.Line(width=3),
            showlegend=True,
            name=x)
    )


fig.update_layout(
    font_family="Epistemix Label",
    yaxis_title="Count",
    xaxis_title="Sim day",
    legend_title="Agent status",
    title="Rebellion model, geospatial implementation",
    title_font_size=24,
    hovermode="x unified",height=400,
)

fig.show()

In [ ]:
sim_data = sim_data.merge(place_agents.rename(columns={"ID":'place_id'}).drop(columns=["agent_type","x_pos","y_pos"]), 
               on="place_id", 
               how="left")

anim_places = pd.DataFrame()
for i in range(sim_data.simday.max()+1):
    tmp = pd.DataFrame()
    tmp["place_id"] = place_agents["ID"]
    tmp["simday"] = i
    tmp["lat"] = place_agents["lat"]
    tmp["lon"] = place_agents["lon"]
    
    dict = {"place_id":[-5,-4,-3,-2,-1], "simday":[i]*5, 
            "lat":[center[1]]*5, "lon":[center[0]]*5,}
    
    anim_places = pd.concat([anim_places, pd.DataFrame(dict), tmp], ignore_index=True)

anim_places = anim_places.merge(sim_data[["id","simday", "agent_type", "agent_status", "place_id"]], 
                                on=["place_id", "simday"], how="left")

anim_places = anim_places.assign(size_val=3.0)

anim_places.loc[anim_places.place_id==-5, "agent_status"] = "Empty"
anim_places.loc[anim_places.place_id==-4, "agent_status"] = "Jailed"
anim_places.loc[anim_places.place_id==-3, "agent_status"] = "Passive"
anim_places.loc[anim_places.place_id==-2, "agent_status"] = "Rebel"
anim_places.loc[anim_places.place_id==-1, "agent_status"] = "Cop"

anim_places.id = anim_places.id.fillna(0)
anim_places.agent_status = anim_places.agent_status.fillna("Empty")
anim_places.agent_type = anim_places.agent_type.fillna(1)

anim_places.loc[anim_places.agent_status=="Empty", "size_val"] = 0.8

In [ ]:
fig = px.scatter_mapbox(anim_places, 
                        lon="lon", lat="lat", 
                        color="agent_status",
                        color_discrete_sequence=[
                            "white", # empty
                            "gray", # jailed
                            px.colors.qualitative.Plotly[2], # passive
                            px.colors.qualitative.Plotly[1], # rebel
                            px.colors.qualitative.Plotly[0]], # cop
                        opacity=0.95,
                        zoom=9.6,
                        size="size_val", size_max=5,
                        animation_frame="simday",
                        animation_group="place_id"
                )


fig.update_layout(mapbox_style=mapstyle, mapbox_accesstoken=token)

fig.update_layout(height=800,)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 200
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 0

fig.update_layout(
    font_family="Epistemix Label",
    title="Rebellion model; Grand Isle County, VT",
    title_font_size=24,
)

fig.show()

In [ ]:
epstein_rebellion_geo_job.delete(interactive=False)